# Credit Risk Scorecard Model Development

## Introduction

The **Credit risk Scorecard** model created from the Lending Club dataset is instrumental in computing the Probability of Default (PD), a key factor in ECL calculations. This scorecard assesses several credit characteristics of potential borrowers, like their credit history, income, outstanding debts, and more, each of which is assigned a specific score. By combining these scores, we derive a total score for each borrower, which translates into an estimated Point-in-Time (PiT) PD. The PiT PD reflects the borrower's likelihood of default at a specific point in time, accounting for both current and foreseeable future conditions.

Additionally, for a holistic view of credit risk, it's essential to estimate the Lifetime PD. The Lifetime PD, as the name suggests, predicts the borrower's likelihood of default throughout the life of the exposure, taking into account potential future changes in the economic and financial conditions.

## Setup

### Import Libraries

In [1]:
from notebooks.probability_of_default.helpers.Developer import Developer
from notebooks.probability_of_default.helpers.scorecard_model import *
from notebooks.probability_of_default.helpers.model_development import *

from IPython.display import HTML


### Input Parameters

In [2]:
default_column = "default"

lending_club_url = "https://vmai.s3.us-west-1.amazonaws.com/datasets/lending_club_loan_data_2007_2014.csv"

preliminary_features_to_drop = [
    "Unnamed: 0",
    "id", "member_id", "funded_amnt", "emp_title", "url", "desc", "application_type",
    "title", "zip_code", "delinq_2yrs", "mths_since_last_delinq", "mths_since_last_record",
    "revol_bal", "total_rec_prncp", "total_rec_late_fee", "recoveries", "out_prncp_inv", "out_prncp", 
    "collection_recovery_fee", "next_pymnt_d", "initial_list_status", "pub_rec",
    "collections_12_mths_ex_med", "policy_code", "acc_now_delinq", "pymnt_plan",
    "tot_coll_amt", "tot_cur_bal", "total_rev_hi_lim", "last_pymnt_d", "last_credit_pull_d",
    'earliest_cr_line', 'issue_d']

final_features_to_drop = ['addr_state', 'total_rec_int', 'loan_amnt',
                    'funded_amnt_inv', 'dti', 'revol_util', 'total_pymnt', 
                    'total_pymnt_inv', 'last_pymnt_amnt', "inq_last_6mths"]

min_missing_percentage = 0.8

iqr_threshold = 1.5

test_size = 0.2

woe_breaks_adj = {"int_rate": [5,10,15]}

# Import Data 
id_params = {"source": lending_club_url}

# Data Preparation 
dp_params = {
    "features_to_drop": preliminary_features_to_drop,
    "default_column": default_column,
    "min_missing_percentage": min_missing_percentage,
    "iqr_threshold": iqr_threshold,
}

# Data Split
ds_params = {
    "target_column": default_column,
    "test_size": test_size
}

# Feature Selection
fs_params = {
    "features_to_drop": final_features_to_drop
}

# Feature Engineering 
fe_params = {
    "target_column": default_column,
    "woe_breaks_adj": woe_breaks_adj
}

# Model Training 
mt_params = {
    "target_column": default_column,
    "add_constant": False
}

## Model Development

In [3]:
df_raw = import_data(id_params)

/Users/juanvalidmind/Dev/github/validmind-python/notebooks/probability_of_default/helpers/scorecard_model.py:15: DtypeWarning: Columns (20) have mixed types. Specify dtype option on import or set low_memory=False.
  df_out = pd.read_csv(source)
INFO: Details of Import Data:
INFO: Columns: ['Unnamed: 0', 'id', 'member_id', 'loan_amnt', 'funded_amnt', 'funded_amnt_inv', 'term', 'int_rate', 'installment', 'grade', 'sub_grade', 'emp_title', 'emp_length', 'home_ownership', 'annual_inc', 'verification_status', 'issue_d', 'loan_status', 'pymnt_plan', 'url', 'desc', 'purpose', 'title', 'zip_code', 'addr_state', 'dti', 'delinq_2yrs', 'earliest_cr_line', 'inq_last_6mths', 'mths_since_last_delinq', 'mths_since_last_record', 'open_acc', 'pub_rec', 'revol_bal', 'revol_util', 'total_acc', 'initial_list_status', 'out_prncp', 'out_prncp_inv', 'total_pymnt', 'total_pymnt_inv', 'total_rec_prncp', 'total_rec_int', 'total_rec_late_fee', 'recoveries', 'collection_recovery_fee', 'last_pymnt_d', 'last_pymnt_

In [4]:
df_prepared = data_preparation(df_raw, dp_params)

features_to_drop: ['Unnamed: 0', 'id', 'member_id', 'funded_amnt', 'emp_title', 'url', 'desc', 'application_type', 'title', 'zip_code', 'delinq_2yrs', 'mths_since_last_delinq', 'mths_since_last_record', 'revol_bal', 'total_rec_prncp', 'total_rec_late_fee', 'recoveries', 'out_prncp_inv', 'out_prncp', 'collection_recovery_fee', 'next_pymnt_d', 'initial_list_status', 'pub_rec', 'collections_12_mths_ex_med', 'policy_code', 'acc_now_delinq', 'pymnt_plan', 'tot_coll_amt', 'tot_cur_bal', 'total_rev_hi_lim', 'last_pymnt_d', 'last_credit_pull_d', 'earliest_cr_line', 'issue_d']
default_column: default
min_missing_percentage: 0.8
iqr_threshold: 1.5

Initial DataFrame:
Number of columns: 75
Columns: Unnamed: 0, id, member_id, loan_amnt, funded_amnt, funded_amnt_inv, term, int_rate, installment, grade, sub_grade, emp_title, emp_length, home_ownership, annual_inc, verification_status, issue_d, loan_status, pymnt_plan, url, desc, purpose, title, zip_code, addr_state, dti, delinq_2yrs, earliest_cr_lin

In [5]:
df_train, df_test = data_split(df_prepared, ds_params)

INFO: Details of Training Data:
INFO: Columns: ['loan_amnt', 'funded_amnt_inv', 'term', 'int_rate', 'installment', 'grade', 'sub_grade', 'emp_length', 'home_ownership', 'annual_inc', 'verification_status', 'purpose', 'addr_state', 'dti', 'inq_last_6mths', 'open_acc', 'revol_util', 'total_acc', 'total_pymnt', 'total_pymnt_inv', 'total_rec_int', 'last_pymnt_amnt', 'default']
INFO: Number of Rows: 181771
INFO: Number of Columns: 23
INFO: Data Types: {'loan_amnt': dtype('int64'), 'funded_amnt_inv': dtype('float64'), 'term': dtype('O'), 'int_rate': dtype('float64'), 'installment': dtype('float64'), 'grade': dtype('O'), 'sub_grade': dtype('O'), 'emp_length': dtype('float64'), 'home_ownership': dtype('O'), 'annual_inc': dtype('float64'), 'verification_status': dtype('O'), 'purpose': dtype('O'), 'addr_state': dtype('O'), 'dti': dtype('float64'), 'inq_last_6mths': CategoricalDtype(categories=[0.0, 1.0, 2.0, 3.0, 4.0, 5.0, 6.0, 7.0, 8.0], ordered=False), 'open_acc': dtype('float64'), 'revol_util

Training data has 181771 rows and 22 columns.
Test data has 45443 rows and 22 columns.


INFO: Total Missing Values: 7097
INFO: Details of Test Data:
INFO: Columns: ['loan_amnt', 'funded_amnt_inv', 'term', 'int_rate', 'installment', 'grade', 'sub_grade', 'emp_length', 'home_ownership', 'annual_inc', 'verification_status', 'purpose', 'addr_state', 'dti', 'inq_last_6mths', 'open_acc', 'revol_util', 'total_acc', 'total_pymnt', 'total_pymnt_inv', 'total_rec_int', 'last_pymnt_amnt', 'default']
INFO: Number of Rows: 45443
INFO: Number of Columns: 23
INFO: Data Types: {'loan_amnt': dtype('int64'), 'funded_amnt_inv': dtype('float64'), 'term': dtype('O'), 'int_rate': dtype('float64'), 'installment': dtype('float64'), 'grade': dtype('O'), 'sub_grade': dtype('O'), 'emp_length': dtype('float64'), 'home_ownership': dtype('O'), 'annual_inc': dtype('float64'), 'verification_status': dtype('O'), 'purpose': dtype('O'), 'addr_state': dtype('O'), 'dti': dtype('float64'), 'inq_last_6mths': CategoricalDtype(categories=[0.0, 1.0, 2.0, 3.0, 4.0, 5.0, 6.0, 7.0, 8.0], ordered=False), 'open_acc': d

In [6]:
df_train_feature_selection = feature_selection(df_train, fs_params)
df_test_feature_selection = feature_selection(df_test, fs_params)

Rows retained with purpose 'debt_consolidation' or 'credit_card': 142293


INFO: Details of Feature Selection:
INFO: Columns: ['term', 'int_rate', 'installment', 'grade', 'sub_grade', 'emp_length', 'home_ownership', 'annual_inc', 'verification_status', 'purpose', 'open_acc', 'total_acc', 'default']
INFO: Number of Rows: 181771
INFO: Number of Columns: 13
INFO: Data Types: {'term': dtype('O'), 'int_rate': dtype('float64'), 'installment': dtype('float64'), 'grade': dtype('O'), 'sub_grade': dtype('O'), 'emp_length': dtype('float64'), 'home_ownership': dtype('O'), 'annual_inc': dtype('float64'), 'verification_status': dtype('O'), 'purpose': dtype('O'), 'open_acc': dtype('float64'), 'total_acc': dtype('float64'), 'default': dtype('int64')}
INFO: Total Missing Values: 6941
INFO: Details of Feature Selection:
INFO: Columns: ['term', 'int_rate', 'installment', 'grade', 'sub_grade', 'emp_length', 'home_ownership', 'annual_inc', 'verification_status', 'purpose', 'open_acc', 'total_acc', 'default']
INFO: Number of Rows: 45443
INFO: Number of Columns: 13
INFO: Data Types

Rows after removing grades 'F' or 'G': 137816
Rows after removing sub_grades starting with 'F' or 'G': 137816
Rows after removing home_ownership values 'OTHER', 'NONE', or 'ANY': 137723
Total rows dropped: 44048
Rows retained with purpose 'debt_consolidation' or 'credit_card': 35532
Rows after removing grades 'F' or 'G': 34349
Rows after removing sub_grades starting with 'F' or 'G': 34349
Rows after removing home_ownership values 'OTHER', 'NONE', or 'ANY': 34322
Total rows dropped: 11121


In [7]:
df_train_feature_eng = feature_engineering(df_train_feature_selection, fe_params)
df_test_feature_eng = feature_engineering(df_test_feature_selection, fe_params)

Performing binning with breaks_adj: {'int_rate': [5, 10, 15]}
[INFO] creating woe binning ...
Converting 12 features to WoE values.
[INFO] converting into woe values ...


INFO: Details of Feature Engineering:
INFO: Columns: ['term', 'int_rate', 'installment', 'grade', 'sub_grade', 'emp_length', 'home_ownership', 'annual_inc', 'verification_status', 'purpose', 'open_acc', 'total_acc', 'default']
INFO: Number of Rows: 181771
INFO: Number of Columns: 13
INFO: Data Types: {'term': dtype('O'), 'int_rate': dtype('float64'), 'installment': dtype('float64'), 'grade': dtype('O'), 'sub_grade': dtype('O'), 'emp_length': dtype('float64'), 'home_ownership': dtype('O'), 'annual_inc': dtype('float64'), 'verification_status': dtype('O'), 'purpose': dtype('O'), 'open_acc': dtype('float64'), 'total_acc': dtype('float64'), 'default': dtype('int64')}
INFO: Total Missing Values: 6941


Successfully converted features to WoE values.
Performing binning with breaks_adj: {'int_rate': [5, 10, 15]}
[INFO] creating woe binning ...
Converting 12 features to WoE values.
[INFO] converting into woe values ...


INFO: Details of Feature Engineering:
INFO: Columns: ['term', 'int_rate', 'installment', 'grade', 'sub_grade', 'emp_length', 'home_ownership', 'annual_inc', 'verification_status', 'purpose', 'open_acc', 'total_acc', 'default']
INFO: Number of Rows: 45443
INFO: Number of Columns: 13
INFO: Data Types: {'term': dtype('O'), 'int_rate': dtype('float64'), 'installment': dtype('float64'), 'grade': dtype('O'), 'sub_grade': dtype('O'), 'emp_length': dtype('float64'), 'home_ownership': dtype('O'), 'annual_inc': dtype('float64'), 'verification_status': dtype('O'), 'purpose': dtype('O'), 'open_acc': dtype('float64'), 'total_acc': dtype('float64'), 'default': dtype('int64')}
INFO: Total Missing Values: 1732


Successfully converted features to WoE values.


In [8]:
df_train_feature_eng = add_constant(df_train_feature_eng)
df_test_feature_eng = add_constant(df_test_feature_eng)

model_fit_candidate = model_training(df_train_feature_eng, mt_params)

Added constant to dataframe. Number of columns went from 13 to 14.
Added constant to dataframe. Number of columns went from 13 to 14.
Training the model with 13 features and 181771 data points.
GLM Logisitic Regression Model trained successfully.


In [9]:
print(model_fit_candidate.summary())

                 Generalized Linear Model Regression Results                  
Dep. Variable:                default   No. Observations:               181771
Model:                            GLM   Df Residuals:                   181758
Model Family:                Binomial   Df Model:                           12
Link Function:                  Logit   Scale:                          1.0000
Method:                          IRLS   Log-Likelihood:                -80653.
Date:                Wed, 23 Aug 2023   Deviance:                   1.6131e+05
Time:                        13:28:04   Pearson chi2:                 1.82e+05
No. Iterations:                     5   Pseudo R-squ. (CS):            0.07327
Covariance Type:            nonrobust                                         
                              coef    std err          z      P>|z|      [0.025      0.975]
-------------------------------------------------------------------------------------------
const                     

## Save Data and Models

In [10]:
developer = Developer()

objects_to_store = {
    "df_raw": df_raw,
    "df_prepared": df_prepared,
    "df_train": df_train,
    "df_train_feature_selection": df_train_feature_selection,
    "df_train_feature_eng": df_train_feature_eng,
    "df_test_feature_eng": df_test_feature_eng,
    "model_fit_final": model_fit_candidate,
}

developer.save_objects_to_pickle(
    filename="datasets/scorecard_data_and_models.pkl", 
    objects_to_save=objects_to_store)

INFO: Saved 7 objects to datasets/scorecard_data_and_models.pkl


['df_raw',
 'df_prepared',
 'df_train',
 'df_train_feature_selection',
 'df_train_feature_eng',
 'df_test_feature_eng',
 'model_fit_final']